In [11]:
#Importing Libraries
import json
import socket
import time
from threading import Thread
import os, sys, shutil, signal
import subprocess
from subprocess import Popen, CREATE_NEW_CONSOLE

In [12]:
#Cleaning the Output Path
directory_path = r"C:\Users\jaske\OneDrive\Desktop\NGO\projects\socket"
shutil.rmtree(directory_path)
os.mkdir(directory_path)

In [13]:
#Starting Tabpy Server
outputpath = directory_path + "\output.txt"

cmd = "tabpy > " + outputpath
print("Running the command: " + str(cmd))
process = Popen(cmd.split(), shell=True, stdin=None, stdout=None, stderr=None, close_fds=True, creationflags=CREATE_NEW_CONSOLE)
process_pid = process.pid

found = 0
while(found == 0):
    if(os.path.isfile(outputpath)):
        time.sleep(0.5)
        print("Output file found")
        found = 1
    else:
        time.sleep(2)

f = open(outputpath)
tabpy_hostname = socket.gethostbyname(socket.gethostname()) + " or localhost"
tabpy_port = int(f.readlines()[-1].split(" ")[-1].strip())
f.close()

print("Tabpy Hostname: " + tabpy_hostname)
print("Tabpy running on the port: " + str(tabpy_port))

Running the command: tabpy > C:\Users\jaske\OneDrive\Desktop\NGO\projects\socket\output.txt
Output file found
Tabpy Hostname: 192.168.1.42 or localhost
Tabpy running on the port: 9004


In [14]:
#Setting up Socket Server configurations
server_host = socket.gethostbyname(socket.gethostname())
server_port = 9999

f = open(directory_path + "\config.json", 'w')
server_config = {"host":server_host, "port":server_port}
json.dump(server_config, f)
f.close()

In [15]:
#Stopping Tabpy Server
def stop_tabpy(process_pid):
    try:
        subprocess.check_call('taskkill /F /T /PID %i' % process_pid)
        return "Tabpy Server closed successfully"
    except Exception as e:
        return "Error closing Tabpy Server " + str(e)

#Handling Client responses
def send_info_to_client(client, address, tabpy_hostname, tabpy_port, process_pid):
    
    client.send(str.encode("Connected to server, send command like: tabpy/stop_tabpy/quit/stop_server"))
    
    connected = True
    while(connected):
        msg = client.recv(1024).decode("utf-8")
        if((msg == "quit")):
            send_msg = f"{str(address)} disconnected from server"
            connected = False
        elif(msg == "stop_server"):
            global server_status
            server_status = 1
            send_msg = "Socket Server disconnected"
        elif(msg == "stop_tabpy"):
            send_msg = stop_tabpy(process_pid)
        elif("tabpy" in msg):
            send_msg = f"USE hostname: {tabpy_hostname}, port: {tabpy_port}"
        else:
            send_msg = "command not found"
        client.send(str.encode(send_msg))
    print(send_msg)
    client.close()


server = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
server.bind((server_host, server_port))
server.listen(100)
print("Server Started \nServer Binded \nServer Listening \nServer Waiting...")

server_status = 0
while(server_status == 0):
    
    client, address = server.accept()
    print(f"Server Connected to {str(address)} client")
    
    t = Thread(target=send_info_to_client, args=(client, address, tabpy_hostname, tabpy_port, process.pid))
    t.start()
    #t.join()
    
    if(server_status == 1):
        print("Server is offline")
        break
    else:
        print("Server is still online")


print("Closing Server Socket")
server.close()

Server Started 
Server Binded 
Server Listening 
Server Waiting...
Server is still online
Server Connected to ('192.168.1.42', 50084) client
Server is still online
Server Connected to ('192.168.1.155', 39748) client
Server is still online
('192.168.1.42', 50084) disconnected from server
('192.168.1.155', 39748) disconnected from server
Server Connected to ('192.168.1.42', 50105) client
Closing Server Socket
('192.168.1.42', 50105) disconnected from server
